In [28]:
import mysql.connector
import pandas as pd
from mysql.connector import Error

# Database Configuration
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "university"
}



# ძირითადი ფუნქციები

In [93]:
class CustomORM:
    def __init__(self, config):
        self.connection = self.connect_to_database(config)
        self.cursor = self.connection.cursor(dictionary=True)

    
    def connect_to_database(self, config):
        """Connect to the database."""
        try:
            connection = mysql.connector.connect(
                host=config["host"],
                user=config["user"],
                password=config["password"],
                database=config["database"]
            )
            if connection.is_connected():
                print(f"Connected to the database: {config['database']}")
            return connection
        except Error as e:
            print(f"Error: {e}")
            return None

    def create_table(self, table_name, schema):
        """Create a table with the given schema."""
        try:
            self.cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({schema})")
            self.connection.commit()
            print(f"Table {table_name} created successfully.")
        except Error as e:
            print(f"Error: {e}")

    def insert_data(self, table_name, columns, values):
        """Insert single or multiple rows of data into the table."""
        try:
            query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(values[0]))})"
            self.cursor.executemany(query, values)
            self.connection.commit()
            print(f"Data inserted into {table_name}.")
        except Error as e:
            print(f"Error: {e}")

    def select_data(self, table_name, columns="*", where=None, limit=None, order_by=None, like=None, in_operator=None):
        """Perform SELECT query with optional filtering, ordering, limiting, LIKE and IN."""
        query = f"SELECT {columns} FROM {table_name}"
        
       
        if where:
            query += f" WHERE {where}"

      
        if like:
            query += f" WHERE {like}"

      
        if in_operator:
            query += f" WHERE {in_operator}"

      
        if order_by:
            query += f" ORDER BY {order_by}"

      
        if limit:
            query += f" LIMIT {limit}"

        try:
            self.cursor.execute(query)
            results = self.cursor.fetchall()
            return results
        except Error as e:
            print(f"Error: {e}")
            return []

    def print_table(self, table_name):
        """Fetch and print the entire table as a DataFrame."""
        query = f"SELECT * FROM {table_name}"
        try:
            self.cursor.execute(query)
            data = self.cursor.fetchall()
            df = pd.DataFrame(data)
            print(f"\nTable '{table_name}':")
            print(df)
        except Error as e:
            print(f"Error: {e}")
            
    def add_column(self, table_name, column_definition):
        """Add a column to a table."""
        try:
            self.cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_definition}")
            self.connection.commit()
            print(f"Column added to {table_name}.")
        except Error as e:
            print(f"Error: {e}")

    def remove_column(self, table_name, column_name):
        """Remove a column from a table."""
        try:
            self.cursor.execute(f"ALTER TABLE {table_name} DROP COLUMN {column_name}")
            self.connection.commit()
            print(f"Column {column_name} removed from {table_name}.")
        except Error as e:
            print(f"Error: {e}")

    def update_column_type(self, table_name, column_name, new_type):
        """Change the data type of a column."""
        try:
            self.cursor.execute(f"ALTER TABLE {table_name} MODIFY COLUMN {column_name} {new_type}")
            self.connection.commit()
            print(f"Column {column_name} type changed to {new_type}.")
        except Error as e:
            print(f"Error: {e}")

    def close_connection(self):
        """Close the database connection."""
        if self.connection.is_connected():
            self.cursor.close()
            self.connection.close()
            print("Connection closed.")



# მონაცემებზე მოქმედებები


In [100]:

db = CustomORM(config=DB_CONFIG)


Connected to the database: university


In [54]:



# Create a table
students_schema = """
    student_id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    date_of_birth DATE,
    enrollment_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    major VARCHAR(100)
"""
db.create_table("students", students_schema)



Table students created successfully.


# სტუდენტიების ცხრილის ჩვენება

In [120]:

db.print_table("students")


Table 'students':
    student_id first_name   last_name                          email  \
0            1        ანა        ბანა            anabana@example.com   
1            3       მაკა      წაქაძე       makatsakadze@example.com   
2            4     გიორგი   ბურჯანაძე      michael.brown@example.com   
3            5     სალომე    გიორგიძე   salome.giorgidze@example.com   
4            6     დავითი      ლაბაძე      davit.labadze@example.com   
5            7     ანდრია   ბანძელაძე          andriaban@example.com   
6            8       ბეკა     კიკნაძე       beqakiknadze@example.com   
7            9    მარიამი      წაქაძე    mariamitsakadze@example.com   
8           10       ნინო   ბურჯანაძე   elene.burjanadzr@example.com   
9           11      ელენე    გიორგიძე    elene.giorgidze@example.com   
10          12       ლინა   ვარდანიძე     linavardanidze@example.com   
11          13        ინა    გიორგიძე      ina.giorgidze@example.com   
12          31       ნიკო  დევდარიანი    niko

# მონაცემის შეყვანა ცხრილში

In [58]:
# Insert data into students (single or multiple rows)
students_data = [
    ('გიორგი', 'დევდარიანი', 'გიორგი.devdariani@example.com', '2003-11-11', 'Physics')
]
db.insert_data("students", ["first_name", "last_name", "email", "date_of_birth", "major"], students_data)



Data inserted into students.


# ცხრილიდან 5 სტუდენტის სახელის და გვარის გამოტანა

In [60]:
# SELECT query with LIMIT and ORDER BY clauses
students = db.select_data("students", columns="first_name, last_name", order_by="first_name", limit=5)
print(pd.DataFrame(students))



  first_name   last_name
0        ანა        ბანა
1     ანდრია   ბანძელაძე
2       ბეკა     კიკნაძე
3     გიორგი   ბურჯანაძე
4     გიორგი  დევდარიანი


# ცხრილიდან 4 სტუდენტის სახელის, გვარის და ემაილის გამოტანა გამოტანა


In [68]:
# SELECT query with LIMIT and ORDER BY clauses
students = db.select_data("students", columns="first_name, last_name, email", order_by="first_name", limit=4)
print(pd.DataFrame(students))


  first_name  last_name                      email
0        ანა       ბანა        anabana@example.com
1     ანდრია  ბანძელაძე      andriaban@example.com
2       ბეკა    კიკნაძე   beqakiknadze@example.com
3     გიორგი  ბურჯანაძე  michael.brown@example.com


# გამოიტანს მონაცემებს, რომელთა ემაილში სიტყვათაშეთანხმება 'an' არის

In [76]:

students_like = db.select_data("students", columns="first_name, last_name, email", like="email LIKE '%an%'")
print(pd.DataFrame(students_like))



  first_name   last_name                          email
0        ანა        ბანა            anabana@example.com
1     ანდრია   ბანძელაძე          andriaban@example.com
2       ნინო   ბურჯანაძე   elene.burjanadzr@example.com
3       ლინა   ვარდანიძე     linavardanidze@example.com
4       ნიკო  დევდარიანი    niko.devdariani@example.com
5     გიორგი  დევდარიანი  გიორგი.devdariani@example.com


# ფილტრავს მონაცემებს ისე, რომ აირჩიოს მხოლოდ ის სტუდენტები, რომელთა major არის ან 'Mathematics', ან 'Physics' და გამოაქვს მათი სახელი და გვარი

In [40]:
# Using IN operator to filter by major
students_in = db.select_data("students", columns="first_name, last_name, major", in_operator="major IN ('Mathematics', 'Physics')")
print(pd.DataFrame(students_in))



  first_name   last_name        major
0        ანა        ბანა  Mathematics
1       მაკა      წაქაძე      Physics
2     სალომე    გიორგიძე  Mathematics
3     დავითი      ლაბაძე      Physics
4       ბეკა     კიკნაძე  Mathematics
5    მარიამი      წაქაძე      Physics
6      ელენე    გიორგიძე  Mathematics
7        ინა    გიორგიძე  Mathematics
8       ნიკო  დევდარიანი      Physics


# სტუდენტების მონაცემების გამოტანა გვარის მიხედვით

In [102]:

students = db.select_data("students", columns="last_name, major", order_by="last_name ASC")
print(pd.DataFrame(students))


     last_name             major
0         ბანა       Mathematics
1    ბანძელაძე  Computer Science
2    ბურჯანაძე  Computer Science
3    ბურჯანაძე  Computer Science
4     გიორგიძე       Mathematics
5     გიორგიძე       Mathematics
6     გიორგიძე       Mathematics
7   დევდარიანი           Physics
8   დევდარიანი           Physics
9    ვარდანიძე  Computer Science
10     კიკნაძე       Mathematics
11      ლაბაძე           Physics
12      წაქაძე           Physics
13      წაქაძე           Physics


# ახალი სვეტის დამატება ტელეფონის ნომრის ბაზისთვის

In [111]:

db.add_column("students", "phone_number VARCHAR(20)")

db.print_table("students")

Column added to students.

Table 'students':
    student_id first_name   last_name                          email  \
0            1        ანა        ბანა            anabana@example.com   
1            3       მაკა      წაქაძე       makatsakadze@example.com   
2            4     გიორგი   ბურჯანაძე      michael.brown@example.com   
3            5     სალომე    გიორგიძე   salome.giorgidze@example.com   
4            6     დავითი      ლაბაძე      davit.labadze@example.com   
5            7     ანდრია   ბანძელაძე          andriaban@example.com   
6            8       ბეკა     კიკნაძე       beqakiknadze@example.com   
7            9    მარიამი      წაქაძე    mariamitsakadze@example.com   
8           10       ნინო   ბურჯანაძე   elene.burjanadzr@example.com   
9           11      ელენე    გიორგიძე    elene.giorgidze@example.com   
10          12       ლინა   ვარდანიძე     linavardanidze@example.com   
11          13        ინა    გიორგიძე      ina.giorgidze@example.com   
12          31     

# ტელეფონის ნომრის სვეტის წაშლა

In [113]:
db.remove_column("students", "phone_number")


Column phone_number removed from students.


# სახელის ტიპის შეცვლა

In [117]:

db.update_column_type("students", "first_name", "VARCHAR(55)")

Column first_name type changed to VARCHAR(55).


# კავშირის დახურვა

In [80]:

db.close_connection()

Connection closed.
